In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards (104 cards total)
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_deck(deck):
    """Print the entire deck to show all cards in both identical sets."""
    for card in deck:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        print(f"{rank_name} of {suit_name}", end=", ")
    print("\n")

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """
    Shuffle the deck and select five unique cards.
    """
    shuffled_deck = np.random.permutation(deck)
    hand_indices = np.random.choice(len(shuffled_deck), 5, replace=False)
    hand = shuffled_deck[hand_indices]
    sorted_hand = np.sort(hand, order='rank')
    return shuffled_deck, sorted_hand

def is_straight(hand):
    """Check if a hand is a regular Straight (not a straight flush or royal flush)."""
    # Extract ranks and sort them
    ranks_in_hand = sorted(set(card['rank'] for card in hand))  # Unique ranks only

    # Extract suits
    suits_in_hand = [card['suit'] for card in hand]

    # Check if we have exactly five unique consecutive ranks and not all cards have the same suit
    if len(ranks_in_hand) == 5 and (max(ranks_in_hand) - min(ranks_in_hand) == 4) and len(set(suits_in_hand)) > 1:
        return True
    return False



def run_simulation(num_trials, worker_id=None):

    if worker_id is not None:
        # Create unique seed using large prime numbers and worker id
        base_seed = 104729  # Large prime
        multiplier = 15485863  # Another large prime
        worker_seed = base_seed + (worker_id * multiplier)
        np.random.seed(worker_seed)


    straight_flush_count = 0
    local_examples = []

    # Process hands in smaller batches for better progress tracking
    batch_size = 100_000
    num_batches = num_trials // batch_size + (1 if num_trials % batch_size else 0)

    for batch in range(num_batches):
        current_batch_size = min(batch_size, num_trials - batch * batch_size)

        for _ in range(current_batch_size):
            _, hand = deal_hand(deck)
            if is_straight(hand):
                straight_count += 1
                if len(local_examples) < 10:
                    local_examples.append(hand.copy())

        if worker_id is not None:
            print(f"Worker {worker_id}: Completed batch {batch + 1}/{num_batches}, "
                  f"found {straight_count} Straight Flushes so far")

    return straight_count, local_examples

def run_simulation(num_trials, worker_id=None):
    if worker_id is not None:
        # Create unique seed using large prime numbers and worker id
        base_seed = 104729  # Large prime
        multiplier = 15485863  # Another large prime
        worker_seed = base_seed + (worker_id * multiplier)
        np.random.seed(worker_seed)

    # Initialize the counter for Straight hands
    straight_count = 0  # Initialize to zero at the beginning
    local_examples = []

    # Process hands in smaller batches for better progress tracking
    batch_size = 100_000
    num_batches = num_trials // batch_size + (1 if num_trials % batch_size else 0)

    for batch in range(num_batches):
        current_batch_size = min(batch_size, num_trials - batch * batch_size)

        for _ in range(current_batch_size):
            _, hand = deal_hand(deck)
            if is_straight(hand):
                straight_count += 1  # Increment the counter if it's a Straight
                if len(local_examples) < 10:
                    local_examples.append(hand.copy())

        if worker_id is not None:
            print(f"Worker {worker_id}: Completed batch {batch + 1}/{num_batches}, "
                  f"found {straight_count} Straight Flushes so far")

    return straight_count, local_examples




    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Distribute work with adjusted chunks
        futures = []
        for i in range(num_workers):
            # Last worker gets remaining hands if any
            worker_trials = chunk_size + (remaining if i == num_workers-1 else 0)
            futures.append(executor.submit(run_simulation, worker_trials, worker_id=i))

        # Track progress and collect results
        completed = 0
        results = []
        for future in concurrent.futures.as_completed(futures):
            completed += 1
            sf_count, examples = future.result()
            results.append((sf_count, examples))

    # Simply combine all results without checking for duplicates
    for sf_count, examples in results:
        total_straight_count += sf_count
        all_examples.extend(examples)  # Add all examples, including duplicates

    # Print final summary

    print(f"Total example hands collected: {len(all_examples)}")

    return total_straight_count, all_examples

def calculate_probability(straight_count, total_hands):
    """Calculate the probability of getting a Straight."""
    return straight_count / total_hands

def main():
    # Print the initial unshuffled deck
    print("Initial deck:")
    print_deck(deck)

    # Part 1: Run 100 trials, shuffle each time, and print the shuffled deck and dealt hand
    print("\nRunning 100 trials:")
    for trial in range(100):
        shuffled_deck, hand = deal_hand(deck)  # Shuffle and deal a hand
        print(f"\nTrial {trial + 1}: Shuffled deck")
        print_deck(shuffled_deck)  # Print the entire shuffled deck after shuffling
        print_hand(hand)
        if is_straight(hand):
            print("Straight!")

    # Part 2: Run full simulation for larger trial counts and calculate Straight Flush probability
    trial_counts = [1_00, 1_000, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        straight_flush_count, example_hands_large = run_simulation_parallel(trials)

        # Calculate total probability for Straight Flush hands
        total_probability = calculate_probability(straight_flush_count, trials)

        # Print results and total probability for Straight Flush
        print(f"\nResults for {trials} trials:")
        print(f"Total Straight  occurrences: {straight_flush_count}")
        print(f"Straight probability: {total_probability:.8f}")

        # Only print example hands for 1,000,000 trials
        if trials == 1_000_000:
            print("\nAll Example Hands of Straight  from the 1,000,000-trial simulation:")
            for hand in example_hands_large:
                print_hand(hand)

if __name__ == "__main__":
    main()



Initial deck:
2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 7 of Clubs, 7 of Spades, 8 of Hearts, 8 of Diamonds, 8 of Clubs, 8 of Spades, 9 of Hearts, 9 of Diamonds, 9 of Clubs, 9 of Spades, 10 of Hearts, 10 of Diamonds, 10 of Clubs, 10 of Spades, J of Hearts, J of Diamonds, J of Clubs, J of Spades, Q of Hearts, Q of Diamonds, Q of Clubs, Q of Spades, K of Hearts, K of Diamonds, K of Clubs, K of Spades, A of Hearts, A of Diamonds, A of Clubs, A of Spades, 2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 